In [1]:
import numpy as np
import torch
import opt_einsum
import matplotlib.pyplot as plt

import utils
import cvxpy_caller
import process_matrix
import error_schemes
import qec_schemes

import importlib
importlib.reload(utils)
importlib.reload(process_matrix)
importlib.reload(cvxpy_caller)
importlib.reload(error_schemes)
importlib.reload(qec_schemes)

<module 'qec_schemes' from '/n/home10/jmcgreivy/convex-quantum-error-correction/qec_schemes.py'>

In [2]:
# might expect a stable range as the error rate approaches zero you'll approach this optimal code
# look into enforcing sparsity of Krauss operators

In [3]:
def plot_find_vs_error(qec, error_scheme, label):
    fids = []
    errors = np.linspace(0, 1, 25)
    for e in errors:
        X_E = error_scheme(int(np.log2(qec["X_R"].shape[0])), e)
        fid = utils.compute_fidelity(X_C=qec["X_C"], X_E=X_E, X_D=qec["X_D"]).item()
        fids.append(fid)
    plt.plot(errors, np.array(fids), label = label)

In [ ]:
cvx_qec_2 = qec_schemes.convex_optimizer_qec(1, 2, error_schemes.GAD(q=2, l=0.5))
plot_find_vs_error(cvx_qec_2, error_schemes.GAD, label="Convex Optimizer 2 qubit ")

cvx_qec_3 = qec_schemes.convex_optimizer_qec(1, 3, error_schemes.GAD(q=3, l=0.5))
plot_find_vs_error(cvx_qec_3, error_schemes.GAD, label="Convex Optimizer 3 qubit")

cvx_qec_4 = qec_schemes.convex_optimizer_qec(1, 4, error_schemes.GAD(q=4, l=0.5))
plot_find_vs_error(cvx_qec_4, error_schemes.GAD, label="Convex Optimizer 4 qubit")

plt.legend()